In [ ]:
import sys, os

# unzipped download from webpage:
dataset_location = "../Datasets/beach-cleaning-object-detection.v1i.yolov5pytorch/"
print("Dataset direcory is:", dataset_location)


yolopath = "../yolov5/" # utils.__file__.replace(os.getcwd()+"/",'').split('utils')[0]
print("yolo base direcoty is:", yolopath)

sys.path.append(yolopath) # add yolo code folder
import utils


In [ ]:
#customize iPython writefile so we can write variables in cells, like {num_classes}
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Read dataset information:

In [ ]:
# read number of classes based on YAML config file:
import yaml
with open(dataset_location + "/data.yaml", 'r') as stream:
    data_cfg = yaml.safe_load(stream)
    num_classes = str(data_cfg['nc'])
    list_classes = data_cfg['names']
    print("Dataset defines", num_classes, "classes ->\n", list_classes)
    

## Update path information for yolo training:

In [ ]:
# read current config:
with open(dataset_location + "/data.yaml", 'r') as stream:
    data_cfg = yaml.safe_load(stream)
    
#modifiy path relative to yolo source folder:
data_cfg['train']=dataset_location + "/train/images"
data_cfg['val']=dataset_location + "/valid/images"
data_cfg['test']=dataset_location + "/test/images"
    
# write modified config:
with open(dataset_location + "/data.yaml", 'w') as file:
    yaml.dump(data_cfg, file)

In [ ]:
#this is the final dataset configuration file:
%cat {dataset_location}/data.yaml

In [ ]:
#this is the model configuration we will use
%cat {yolopath}/models/yolov5s.yaml

## Write model configuration file for yolo training:

In [ ]:
%%writetemplate {yolopath}/models/beachbot_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
## Enter yolo source directory and start training:

In [ ]:
from subprocess import call
eval_img_widths = [1280, 640, 320, 160]
num_epochs = 200

for img_width in eval_img_widths:
    targetfolder = "beachbot_yolov5s_" + str(img_width)
    #check if solder exists -> skip computation!
    if os.path.isdir(yolopath + targetfolder + "_results"):
        print("Skipping run", img_width, ":", targetfolder, "exists!")
    else:
        print("Start run", img_width)
        params=("--img " + str(img_width) + " --batch 16 --epochs " + str(num_epochs) + " --data " + dataset_location + "/data.yaml --cfg " + yolopath + "/models/beachbot_yolov5s.yaml --weights '' --name " + targetfolder + "_results  --cache").split()
        call(["python", yolopath+"train.py"] + params)
